In [7]:
from __future__ import annotations

import random

from combat.runtime_state import *
from combat.magic_menu import *
from combat.char_build import *
from combat.debug_utils import *
from combat.enemy_build import *
from combat.input_ui import *
from combat.battle_sim import *
from combat.enemy_selection import (
    LocationMonsters,
    build_location_index,
    pick_enemy_names,
    danger_label,
    calc_party_avg_level,
)


def choose_location_console(
    entries: list[LocationMonsters], *, party_avg_lv: int
) -> LocationMonsters:
    print("=== 場所を選択してください ===")
    for i, e in enumerate(entries, start=1):
        dg = danger_label(e, party_avg_lv)
        diff = e.avg_level - party_avg_lv

        print(
            f"{i:>3}. {e.location}  "
            f"(monsters: {len(e.monster_names)}) "
            f"(LV: {e.avg_level} / {e.min_level}-{e.max_level}) "
            f"(Δ: {diff:+}) "
            f"(Danger: {dg})"
        )

    while True:
        s = input("番号を入力 > ").strip()
        if s.isdigit():
            idx = int(s)
            if 1 <= idx <= len(entries):
                return entries[idx - 1]
        print(f"1〜{len(entries)} の範囲で数字を入力してください。")

In [9]:
# =========================
# JSON 読み込み
# =========================
state = init_runtime_state()  # runtime_state

# キャラごとのそのジョブで使える魔法一覧（リスト）
party_magic_info = build_party_magic_info(state)  # magic_menu
party_magic_lists = build_party_magic_lists(state)  # magic_menu
# 召喚魔法の子Spellsを展開した辞書
spells_expanded = expand_spells_for_summons(state.spells)  # magic_menu

In [ ]:
# ==================================================
# １．セーブデータ → キャラ最終ステ（パーティ全員）
# ==================================================
level_table = LevelTable("assets/data/level_exp.csv")  # パスは実プロジェクトに合わせて
party_members = build_party_members_from_save(
    save=state.save,
    weapons=state.weapons,
    armors=state.armors,
    jobs_by_name=state.jobs_by_name,
    level_table=level_table,
)  # char_build

print("weapons type:", type(state.weapons), "len:", len(state.weapons))
if isinstance(state.weapons, dict):
    k = next(iter(state.weapons.keys()))
    print(
        "weapons sample key:",
        repr(k),
        "value keys:",
        list(state.weapons[k].keys())[:10],
    )
else:
    print("weapons[0] keys:", list(state.weapons[0].keys())[:10])

print_party_debug_summary(party_members, party_magic_lists)
print_inventory(state.save, show_zero=True)  # debug_utils

[DBG eq] Runeth eq: EquipmentSet(main_hand='Mythril Rod', off_hand='Book of Light', head='Leather Cap', body='Vest', arms='Protect Ring')
[DBG weapon index size] 84 norm size 84
[DBG eq main/off] 'Mythril Rod' / 'Book of Light'
[DBG eq.main_hand] 'Mythril Rod' raw_in_weapons? True
[DBG eq.off_hand ] 'Book of Light' raw_in_weapons? False
[warn] armor not found: Book of Light (norm=bookoflight)
[DBG eq] Refia eq: EquipmentSet(main_hand='Thunder Spear', off_hand='Ice Shield', head='Leather Cap', body='Vest', arms='Mythril Gloves')
[DBG weapon index size] 84 norm size 84
[DBG eq main/off] 'Thunder Spear' / 'Ice Shield'
[DBG eq.main_hand] 'Thunder Spear' raw_in_weapons? True
[DBG eq.off_hand ] 'Ice Shield' raw_in_weapons? False
[warn] weapon not found: Ice Shield (norm=iceshield)
weapons type: <class 'dict'> len: 84
weapons sample key: 'Kaiser Knuckles' value keys: ['name', 'Type', 'BasePower', 'BaseAccuracy', 'Price', 'Value', 'EquippedBy', 'Shops']
[Runeth / Sage]
Lv20/J90 FRONT  HP 747/7

In [13]:
# ==================================================
# ２．敵も複数対応の形に
# ==================================================
# enemy_names = ["Flyer", "Unei'S Clone"]
party_avg_lv = calc_party_avg_level(party_members)
locations = build_location_index(state.monsters)
selected = choose_location_console(locations, party_avg_lv=party_avg_lv)
enemy_names = pick_enemy_names(selected, state.monsters, k_min=2, k_max=6)
enemies = build_enemies(
    enemy_defs_by_name=state.monsters,
    spells_by_name=state.spells,
    enemy_names=enemy_names,
)

print_enemies_status_compact(enemies)  # debug_utils

=== 場所を選択してください ===
  1. Altar Cave B1  (monsters: 4) (LV: 2 / 2-3) (Δ: -18) (Danger: Boss)
  2. Altar Cave B2  (monsters: 3) (LV: 2 / 2-3) (Δ: -18) (Danger: LOW)
  3. Altar Cave B3  (monsters: 4) (LV: 2 / 2-3) (Δ: -18) (Danger: Boss)
  4. Sewers B3  (monsters: 1) (LV: 2 / 2-2) (Δ: -18) (Danger: Boss)
  5. Floating Continent Near Ur  (monsters: 3) (LV: 3 / 3-4) (Δ: -17) (Danger: LOW)
  6. Ur  (monsters: 2) (LV: 3 / 3-3) (Δ: -17) (Danger: LOW)
  7. Dragon's Peak  (monsters: 5) (LV: 5 / 2-8) (Δ: -15) (Danger: Boss)
  8. Sealed Cave B1  (monsters: 3) (LV: 5 / 5-6) (Δ: -15) (Danger: LOW)
  9. Mythril Mines B1  (monsters: 2) (LV: 6 / 5-6) (Δ: -14) (Danger: LOW)
 10. Mythril Mines B2  (monsters: 5) (LV: 6 / 5-7) (Δ: -14) (Danger: LOW)
 11. Sealed Cave B2  (monsters: 4) (LV: 6 / 5-6) (Δ: -14) (Danger: LOW)
 12. Tower of Owen 1F  (monsters: 3) (LV: 6 / 5-6) (Δ: -14) (Danger: LOW)
 13. Tower of Owen 2F  (monsters: 3) (LV: 6 / 5-6) (Δ: -14) (Danger: LOW)
 14. Tower of Owen Base  (monsters: 3) (L

番号を入力 >  7


[Bahamut/1]
Lv2/J49  HP 60000/60000
ATK7(x3) ACC65% | DEF8(x3) EVA50% | MDEF7(x3) RES20%
Status:
  Immune: Poison, Blind, Mini, Silence, Toad, Petrification, KO, Confusion, Sleep, Paralysis, Partial Petrification
Drops:
  Steal: Potion x1.00
------------------------------------------------------------


In [ ]:
# ==================================================
# ３．戦闘ターン
# ==================================================
rng = random.Random()
max_turns = 50

# simulate_one_round_multi_party を1ターンずつ呼び出す場合
for turn in range(1, max_turns + 1):
    print_round_header_and_state(turn, party_members, enemies)    # debug_utils

    # ラウンド前の終了判定
    pre_end = check_battle_end_before_round(party_members, enemies)    # debug_utils
    if pre_end is not None:
        print_end_reason(pre_end)    # debug_utils
        break

    # ① 行動入力
    planned_actions = ask_actions_for_party(    # input_ui
        party_members=party_members,
        enemies=enemies,
        spells_by_name=spells_expanded,
        items_by_name=state.items_by_name,
        party_magic_lists=party_magic_lists,
        save=state.save,
    )

    # デバッグ表示（必要な時だけ呼ぶ運用でもOK）
    print_planned_actions(party_members, planned_actions)    # debug_utils

    # ② イニシアティブ計算＆行動解決
    logs, round_result, _event = simulate_one_round_multi_party(    # battle_sim
        party_members,
        enemies,
        planned_actions,
        rng=rng,
        save=state.save,
        spells_by_name=spells_expanded,
        items_by_name=state.items_by_name,
        state=state
    )
    
    print_logs(logs)    # debug_utils
    
    # ラウンド後の終了判定
    if round_result.end_reason != "continue":
        print_end_reason(round_result.end_reason)    # debug_utils
        break


=== Turn 1 ===
味方:
  Runeth: 747/747
  Refia: 535/535
敵:
  Bahamut/1: 60000/60000

Runeth の行動を選んでください。
Runeth: 747/747
  1: Fight
  2: Magic
  3: Run
  4: Item
